In [1]:
import pandas as pd
import numpy as np
from skbio.stats.composition import clr_inv as softmax
import warnings
import os
os.chdir("../../data/recipients/")
warnings.filterwarnings("ignore")

<frozen importlib._bootstrap>:219: RuntimeWarning: scipy._lib.messagestream.MessageStream size changed, may indicate binary incompatibility. Expected 56 from C header, got 64 from PyObject


# Recipient

In [2]:
ranks_metafeces = pd.read_table('mmvec/metagenome_feces/latent_dim_3_input_prior_1.00_output_prior_1.00_beta1_0.90_beta2_0.95_ranks.txt')
ranks_metafeces = pd.read_table('mmvec/metagenome_feces/latent_dim_3_input_prior_1.00_output_prior_1.00_beta1_0.90_beta2_0.95_ranks.txt')
ranks_metaserum = pd.read_table('mmvec/metagenome_serum/latent_dim_3_input_prior_1.00_output_prior_1.00_beta1_0.90_beta2_0.95_ranks.txt')
ranks_metafx = pd.read_table('mmvec/metagenome_fx_prepost/latent_dim_3_input_prior_1.00_output_prior_1.00_beta1_0.90_beta2_0.95_ranks.txt')
ranks_metahx = pd.read_table('mmvec/metagenome_hx_prepost/latent_dim_3_input_prior_1.00_output_prior_1.00_beta1_0.90_beta2_0.95_ranks.txt')
ranks_fecesserum=pd.read_table('mmvec/feces_serum/latent_dim_3_input_prior_1.00_output_prior_1.00_beta1_0.90_beta2_0.95_ranks.txt')
ranks_fecesfx = pd.read_table('mmvec/feces_fx_prepost/latent_dim_3_input_prior_1.00_output_prior_1.00_beta1_0.90_beta2_0.95_ranks.txt')
ranks_feceshx = pd.read_table('mmvec/feces_hx_prepost/latent_dim_3_input_prior_1.00_output_prior_1.00_beta1_0.90_beta2_0.95_ranks.txt')
ranks_serumfx = pd.read_table('mmvec/serum_fx_prepost/latent_dim_3_input_prior_1.00_output_prior_1.00_beta1_0.90_beta2_0.95_ranks.txt')
ranks_serumhx = pd.read_table('mmvec/serum_hx_prepost/latent_dim_3_input_prior_1.00_output_prior_1.00_beta1_0.90_beta2_0.95_ranks.txt')
#ranks_fxhx=pd.read_table('fx_hx_prepost/latent_dim_3_input_prior_1.00_output_prior_1.00_beta1_0.90_beta2_0.95_ranks.txt')

In [3]:
metagenome=pd.read_table("metagenome.tsv",sep="\t",index_col=0)

In [ ]:
df_metafeces = pd.melt(ranks_metafeces, id_vars='featureid', value_vars=ranks_metafeces.columns,ignore_index=True)
df_metaserum = pd.melt(ranks_metaserum, id_vars='featureid', value_vars=ranks_metaserum.columns,ignore_index=True)
df_metafx = pd.melt(ranks_metafx, id_vars='featureid', value_vars=ranks_metafx.columns,ignore_index=True)
df_metahx = pd.melt(ranks_metahx, id_vars='featureid', value_vars=ranks_metahx.columns,ignore_index=True)
df_fecesserum = pd.melt(ranks_fecesserum, id_vars='featureid', value_vars=ranks_fecesserum.columns,ignore_index=True)
df_fecesfx = pd.melt(ranks_fecesfx, id_vars='featureid', value_vars=ranks_fecesfx.columns,ignore_index=True)
df_feceshx = pd.melt(ranks_feceshx, id_vars='featureid', value_vars=ranks_feceshx.columns,ignore_index=True)
df_serumfx = pd.melt(ranks_serumfx, id_vars='featureid', value_vars=ranks_serumfx.columns,ignore_index=True)
df_serumhx = pd.melt(ranks_serumhx, id_vars='featureid', value_vars=ranks_serumhx.columns,ignore_index=True)
#df_fxhx = pd.melt(ranks_fxhx, id_vars='featureid', value_vars=ranks_fxhx.columns,ignore_index=True)

In [ ]:
df_metafeces.columns=["HEAD","TAIL","WEIGHT"]
df_metaserum.columns=["HEAD","TAIL","WEIGHT"]
df_metafx.columns=["HEAD","TAIL","WEIGHT"]
df_metahx.columns=["HEAD","TAIL","WEIGHT"]
df_fecesserum.columns=["HEAD","TAIL","WEIGHT"]
df_fecesfx.columns=["HEAD","TAIL","WEIGHT"]
df_feceshx.columns=["HEAD","TAIL","WEIGHT"]
df_serumfx.columns=["HEAD","TAIL","WEIGHT"]
df_serumhx.columns=["HEAD","TAIL","WEIGHT"]
#df_fxhx.columns=["HEAD","TAIL","WEIGHT"]

In [6]:
k=300
df_metafeces = df_metafeces.sort_values('WEIGHT', ascending=False).iloc[:k]
df_metaserum = df_metaserum.sort_values('WEIGHT', ascending=False).iloc[:k]
df_metafx = df_metafx.sort_values('WEIGHT', ascending=False).iloc[:k]
df_metahx = df_metahx.sort_values('WEIGHT', ascending=False).iloc[:k]
df_fecesserum = df_fecesserum.sort_values('WEIGHT', ascending=False).iloc[:k]
df_fecesfx = df_fecesfx.sort_values('WEIGHT', ascending=False).iloc[:k]
df_feceshx = df_feceshx.sort_values('WEIGHT', ascending=False).iloc[:k]
df_serumfx = df_serumfx.sort_values('WEIGHT', ascending=False).iloc[:k]
df_serumhx = df_serumhx.sort_values('WEIGHT', ascending=False).iloc[:k]

In [7]:
import mygene
def ensembl2gene(gene_list):
    ens=list()
    for i in gene_list:
        ens.append(i.split(".")[0])
    mg = mygene.MyGeneInfo()
    #ginfo = mg.querymany(ens, scopes='ensembl.transcript',returnall=True)
    ginfo = mg.querymany(ens, returnall=True)
    genes=list()
    for i in ginfo['out']:
        try:
            genes.append(i["symbol"])
        except KeyError:
            genes.append(i["query"])
    return genes

def trimMetagenome(meta_list):
    new_meta=list()
    for m in meta_list:
        new_meta.append(m.split(":")[0])
    return new_meta

In [8]:
#Change ensembleid to gene name ensemble
df_metafx['HEAD']=ensembl2gene(df_metafx['HEAD'])
df_metahx['HEAD']=ensembl2gene(df_metahx['HEAD'])
df_fecesfx['HEAD']=ensembl2gene(df_fecesfx['HEAD'])
df_feceshx['HEAD']=ensembl2gene(df_feceshx['HEAD'])
df_serumfx['HEAD']=ensembl2gene(df_serumfx['HEAD'])
df_serumhx['HEAD']=ensembl2gene(df_serumhx['HEAD'])

querying 1-300...done.
Finished.
62 input query terms found dup hits:
	[('ENSMUSG00000081170', 5), ('ENSMUSG00000022560', 4), ('ENSMUSG00000051048', 11), ('ENSMUSG00000049
3 input query terms found no hit:
	['ENSMUSG00000102426', 'ENSMUSG00000104389', 'ENSMUSG00000096841']
querying 1-300...done.
Finished.
52 input query terms found dup hits:
	[('ENSMUSG00000042682', 16), ('ENSMUSG00000005683', 11), ('ENSMUSG00000029246', 8), ('ENSMUSG0000008
1 input query terms found no hit:
	['ENSMUSG00000107231']
querying 1-300...done.
Finished.
38 input query terms found dup hits:
	[('ENSMUSG00000101795', 23), ('ENSMUSG00000062006', 15), ('ENSMUSG00000045382', 16), ('ENSMUSG000001
querying 1-300...done.
Finished.
50 input query terms found dup hits:
	[('ENSMUSG00000055134', 12), ('ENSMUSG00000005802', 8), ('ENSMUSG00000020806', 19), ('ENSMUSG0000004
querying 1-300...done.
Finished.
43 input query terms found dup hits:
	[('ENSMUSG00000015656', 14), ('ENSMUSG00000034912', 37), ('ENSMUSG00000106472', 2

In [9]:
def createnode(df,color=["feces","metagenome"]):
    color2shape={
            "metagenome":"ellipse",
            "feces":"ellipse",
            "serum":"ellipse",
                "hx":"ellipse",
                "fx":"ellipse",
    }
    node=pd.DataFrame(columns=['NODE','LABEL','COLOR','SIZE','SHAPE','DESC'])
    for i in df["HEAD"]:
        node=node.append({
            'NODE':i,
            'LABEL':i,
            'COLOR':color[0],
            'SIZE':'20',
            'SHAPE':color2shape[color[0]], 
            'DESC': ''
        },ignore_index=True)

    for j in df["TAIL"]:
        split=j.split(":")
        if(len(split)>1 and color[1]=="metagenome"):
            node=node.append({
                'NODE':j.split(":")[0],
                'LABEL':j.split(":")[0],
                'COLOR':color[1],
                'SIZE':'20',
                'SHAPE':color2shape[color[1]],
                #'DESC': j
                'DESC': j.split(":")[1]
            },ignore_index=True)
        else:
            node=node.append({
                'NODE':j,
                'LABEL':j,
                'COLOR':color[1],
                'SIZE':'20',
                'SHAPE':color2shape[color[1]],
                'DESC':''
            },ignore_index=True)
    
    return node

In [10]:
node_metafeces=createnode(df_metafeces,color=["feces","metagenome"])
node_metaserum=createnode(df_metaserum,color=["serum","metagenome"])
node_metafx=createnode(df_metafx,color=["fx","metagenome"])
node_metahx=createnode(df_metahx,color=["hx","metagenome"])
node_fecesserum=createnode(df_fecesserum,color=["serum","feces"])
node_fecesfx=createnode(df_fecesfx,color=["fx","feces"])
node_feceshx=createnode(df_feceshx,color=["hx","feces"])
node_serumfx=createnode(df_serumfx,color=["fx","serum"])
node_serumhx=createnode(df_serumhx,color=["hx","serum"])

In [11]:
metagenomefeces_pwy=set()
metagenomeserum_pwy=set()
metagenomefx_pwy=set()
metagenomehx_pwy=set()

for index, row in node_metafeces[node_metafeces["COLOR"]=="metagenome"].iterrows():
    metagenomefeces_pwy.add(row["NODE"])
for index, row in node_metaserum[node_metaserum["COLOR"]=="metagenome"].iterrows():
    metagenomeserum_pwy.add(row["NODE"])
for index, row in node_metafx[node_metafx["COLOR"]=="metagenome"].iterrows():
    metagenomefx_pwy.add(row["NODE"])
for index, row in node_metahx[node_metahx["COLOR"]=="metagenome"].iterrows():
    metagenomehx_pwy.add(row["NODE"])

In [12]:
df_metafeces_taxa=pd.DataFrame(columns=["HEAD","TAIL","WEIGHT"])
node_metafeces_taxa=pd.DataFrame(columns=["NODE","LABEL","COLOR","SIZE","SHAPE","DESC"])

df_metaserum_taxa=pd.DataFrame(columns=["HEAD","TAIL","WEIGHT"])
node_metaserum_taxa=pd.DataFrame(columns=["NODE","LABEL","COLOR","SIZE","SHAPE","DESC"])

df_metafx_taxa=pd.DataFrame(columns=["HEAD","TAIL","WEIGHT"])
node_metafx_taxa=pd.DataFrame(columns=["NODE","LABEL","COLOR","SIZE","SHAPE","DESC"])

df_metahx_taxa=pd.DataFrame(columns=["HEAD","TAIL","WEIGHT"])
node_metahx_taxa=pd.DataFrame(columns=["NODE","LABEL","COLOR","SIZE","SHAPE","DESC"])

In [13]:
#feces
for m in metagenome.index:
    if "UNINTEGRATED" not in m and "unclassified" not in m:
        pwy=m.split(":")[0]
        if pwy in metagenomefeces_pwy:
            splitpipe=m.split("|")
            if len(splitpipe)==2:
                species=splitpipe[1].split(".")[1]
                row_edge={"HEAD":pwy,"TAIL":species,"WEIGHT":1}
                row_node={"NODE":species,"LABEL":species,"COLOR":"taxa","SIZE":20,"SHAPE":"ellipse","DESC":""}
                df_metafeces_taxa=df_metafeces_taxa.append(row_edge,ignore_index=True)
                node_metafeces_taxa=node_metafeces_taxa.append(row_node,ignore_index=True)
#serum                
for m in metagenome.index:
    if "UNINTEGRATED" not in m and "unclassified" not in m:
        pwy=m.split(":")[0]
        if pwy in metagenomeserum_pwy:
            splitpipe=m.split("|")
            if len(splitpipe)==2:
                species=splitpipe[1].split(".")[1]
                row_edge={"HEAD":pwy,"TAIL":species,"WEIGHT":1}
                row_node={"NODE":species,"LABEL":species,"COLOR":"taxa","SIZE":20,"SHAPE":"ellipse","DESC":""}
                df_metaserum_taxa=df_metaserum_taxa.append(row_edge,ignore_index=True)
                node_metaserum_taxa=node_metaserum_taxa.append(row_node,ignore_index=True)

                
#fx
for m in metagenome.index:
    if "UNINTEGRATED" not in m and "unclassified" not in m:
        pwy=m.split(":")[0]
        if pwy in metagenomefx_pwy:
            splitpipe=m.split("|")
            if len(splitpipe)==2:
                species=splitpipe[1].split(".")[1]
                row_edge={"HEAD":pwy,"TAIL":species,"WEIGHT":1}
                row_node={"NODE":species,"LABEL":species,"COLOR":"taxa","SIZE":20,"SHAPE":"ellipse","DESC":""}
                df_metafx_taxa=df_metafx_taxa.append(row_edge,ignore_index=True)
                node_metafx_taxa=node_metafx_taxa.append(row_node,ignore_index=True)
                
#hx
for m in metagenome.index:
    if "UNINTEGRATED" not in m and "unclassified" not in m:
        pwy=m.split(":")[0]
        if pwy in metagenomehx_pwy:
            splitpipe=m.split("|")
            if len(splitpipe)==2:
                species=splitpipe[1].split(".")[1]
                row_edge={"HEAD":pwy,"TAIL":species,"WEIGHT":1}
                row_node={"NODE":species,"LABEL":species,"COLOR":"taxa","SIZE":20,"SHAPE":"ellipse","DESC":""}
                df_metahx_taxa=df_metahx_taxa.append(row_edge,ignore_index=True)
                node_metahx_taxa=node_metahx_taxa.append(row_node,ignore_index=True)


In [18]:
df_metafeces["TAIL"]=trimMetagenome(df_metafeces["TAIL"])
df_metaserum["TAIL"]=trimMetagenome(df_metaserum["TAIL"])
df_metafx["TAIL"]=trimMetagenome(df_metafx["TAIL"])
df_metahx["TAIL"]=trimMetagenome(df_metahx["TAIL"])

In [ ]:
edge=pd.concat([df_metafeces,df_metaserum,df_fecesserum,df_metafx,df_metafx_taxa,df_metahx,df_metahx_taxa]).drop_duplicates()
node=pd.concat([node_metafeces,node_metaserum,node_fecesserum,node_metafx,node_metafx_taxa,node_metahx,node_metahx_taxa]).drop_duplicates()

In [19]:
edge.to_csv("mmvec/edge_taxa500.tsv",sep="\t",index=False)
node.to_csv("mmvec/node_taxa500.tsv",sep="\t",index=False)